In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

True

In [ ]:
file = 'users_ai-for-earthquake-response_19.xlsx'
users = pd.read_excel(file)

# users

In [3]:
users['Applicable?'].unique()

array(['Yes', 'No'], dtype=object)

In [4]:
valid_users = users[users['Applicable?'] == 'Yes']

valid_users.shape

(142, 10)

In [5]:
invalid_users = users[users['Applicable?'] == 'No']

invalid_users.shape

(135, 10)

In [6]:
api_url = 'https://api.eotdl.com'
dataset_ids = ['685fdfd4acc05f06ec9919e5', '687b5f9825bb57c11769bb8d'] 
# ai-for-earthquake-response (slim and full)

eotdl_responses_slim, eotdl_responses_full = [], []
for _, user in tqdm(users.iterrows(), total=len(users)):
    if user['Applicable?'] != 'Yes':
        eotdl_responses_slim.append('not added')
        eotdl_responses_full.append('not added')        
        continue
    # print(user.Email)
    for ix, dataset_id in enumerate(dataset_ids):
        response = requests.patch(
            f"{api_url}/datasets/{dataset_id}/allow-user", 
            headers={"X-API-Key": os.getenv('KEY')},
            json={"email": user.Email}
        )
        # print(response.status_code)
        # print(response.json())
        if response.status_code == 200:
            if ix == 0:
                eotdl_responses_slim.append('added')
            else:
                eotdl_responses_full.append('added')
        else:
            if ix == 0:
                eotdl_responses_slim.append(response.json()['detail'])
            else:
                eotdl_responses_full.append(response.json()['detail'])
    # break

100%|██████████| 277/277 [00:53<00:00,  5.15it/s]


In [7]:
users['EOTDL Response (slim)'] = eotdl_responses_slim
users['EOTDL Response (full)'] = eotdl_responses_full

# users

In [ ]:
users.to_excel(file.replace('.xlsx', '_eotdl.xlsx'), index=False)